# Linear Regression

> Ce notebook utilise les templates fournis durant les exercices d'application.

In [1]:
import numpy as np
import scipy
from sklearn.metrics import mean_squared_error
from sklearn import datasets
from sklearn.linear_model import LinearRegression as LR
from sklearn.linear_model import Ridge 

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

data = datasets.load_diabetes()
X_train, y_train = data.data, data.target

In [2]:
import scipy as sp

def fit_lu(X, y):
    
    """ Find optimal w using LU decomposition """
    
    L, U = sp.linalg.lu(X.T @ X, permute_l=True)
    b = np.linalg.solve(L, X.T.dot(y))
    return np.linalg.solve(U, b)

In [3]:
w = fit_lu(X_train, y_train)

sk_model = LR(fit_intercept=False)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
print("prediction error: ", error)
assert error <= 1e-12

prediction error:  3.410551082478826e-14


In [4]:
def fit_jacobi(X, y, eps=1e-3):
    
    """ Find optimal w using Jacobi """
    """ return w in the method convergent, None otherwise """
    
    N, d = X.shape
    
    A, b = X.T @ X, X.T @ y
    m = A.diagonal()
    M = np.diag(m)
    N = M - A
    
    if np.linalg.det(M) == 0:
        return None
    
    Mi = np.diag(1/m)
    

    w = np.zeros(d)
    wn = 0.0
    
    Z = Mi @ N
    _, S, _ = np.linalg.svd(Z)
    
    if S[0] >= 1:
        return None
    
    while True:
        wn = Z @ w + Mi @ b
        if np.linalg.norm(wn - w) < eps:
            break
        w = wn
    
    return wn

In [5]:
def fit_gauss_siedel(X, y, eps=1e-3):
    
    """ Find optimal w using Gauss Siedel """
    """ return w in the method convergent, None otherwise """
    
    N, d = X.shape
    
    A, b = X.T @ X, X.T @ y
    D = np.diag(A.diagonal())
    E, F = D - A.copy(), D - A.copy()
    for i in range(A.shape[1]):
        F[i:, :i] = 0
        E[:i, i:] = 0
   
    M = D - E
    N = F
    
    if np.linalg.det(M) == 0:
        return None
    
    Mi = np.linalg.inv(M)

    w = np.zeros(d)
    wn = 0.0

    Z = Mi @ N
    _, S, _ = np.linalg.svd(Z)
    
    if S[0] >= 1:
        return None
    
    while True:
        wn = Z @ w + Mi @ b
        if np.linalg.norm(wn - w) < eps:
            break
        w = wn
    
    return wn

## Combine everything in a class

In [6]:
class LinearRegression():
    def __init__(self, fit_intercept=True, method="lu"):
        self.w = 0
        self.fit_intercept = fit_intercept # bias
        self.method = method
    
    def fit(self, X, y):
        X =  np.hstack( (np.ones((X.shape[0],1)), X) ) if self.fit_intercept else X.copy()
        if self.method == "lu":
            self.w = fit_lu(X, y)
        elif self.method == "jacobi":
            self.w = fit_jacobi(X, y)
        elif self.method == "gauss_siedel":
            self.w = fit_gauss_siedel(X, y)
        
    def predict(self, X):
        X = np.hstack( (np.ones((X.shape[0],1)), X) ) if self.fit_intercept else X.copy()
        return X.dot(self.w)